In [1]:
import math

def compute_movement(current_pos, obj, current_angle_deg): 
    # current_angle_deg = angle between thymio front axis and x-axis
    # Extracting coordinates from current_pos and obj
    current_x, current_y = current_pos
    obj_x, obj_y = obj

    # Calculate distance between current position and the target object
    distance = math.sqrt((obj_x - current_x)**2 + (obj_y - current_y)**2)

    # Calculate angle in radians
    angle_radians = math.atan2(obj_y - current_y, obj_x - current_x)
    #angle_radians = math.atan2(obj_y , obj_x)
    #if (obj_y - current_y) <0 and obj_x - current_x < 0:
    #    angle_radians = -angle_radians
    #print(obj_y - current_y)
    #print(obj_x - current_x)

    # Convert angle to degrees
    angle_degrees = math.degrees(angle_radians) - current_angle_deg

    
    print(f"Distance: {distance}", f"degrees: {angle_degrees}")
    return distance, angle_degrees
    

# Example usage
compute_movement([0, 0], [1, 1], -90)
    

    

Distance: 1.4142135623730951 degrees: 135.0


(1.4142135623730951, 135.0)

In [30]:
#USED TO CHECK FOR COLLISION AT REGULAR INTERVALS, Source : ex week 8
from threading import Timer

class RepeatedTimer(object):
    def __init__(self, interval, function, *args, **kwargs):
        self._timer     = None
        self.interval   = interval
        self.function   = function
        self.args       = args
        self.kwargs     = kwargs
        self.is_running = False
        self.start()

    def _run(self):
        self.is_running = False
        self.start()
        self.function(*self.args, **self.kwargs)

    def start(self):
        if not self.is_running:
            self._timer = Timer(self.interval, self._run)
            self._timer.start()
            self.is_running = True

    def stop(self):
        self._timer.cancel()
        self.is_running = False

In [66]:
from tdmclient import ClientAsync, aw

import time
import numpy as np
import asyncio
Ts = 0.01
SPEED = 300
SEUIL_OBSTACLE = 100
SEUIL_KIDNAPPED = 200
#VALUES THAT NEED TO BE TUNED BEGIN: 
##We add correction factors to tune the speed symmetry
SPEED_LEFT = SPEED +1 
SPEED_RIGHT = SPEED

thymio_speed_to_mms = 19.73913043478261/50 ##TO FIND THE MM/s speed, divide motors_speed by this, thix value has to be tuned for each thymio
rotation_factor = 115*np.pi/360 
#VALUES TO BE TUNED END


obstacle_detected = 0
kidnapped = 0
def get_horizontal():
    global obstacle_detected, kidnapped
    prox_values = node["prox.horizontal"][:5]
    ground_values = node["prox.ground.reflected"]
    #print(list(prox_values))
    #print(list(ground_values))
    if obstacle_detected ==0 :
        obstacle_detected = any(value > SEUIL_OBSTACLE for value in prox_values)

    if kidnapped == 0 :
        if ground_values[0] < SEUIL_KIDNAPPED  or ground_values[1] < SEUIL_KIDNAPPED:
            kidnapped = 1
    


#Adapted from moodle forum thread : 
async def drive_for_seconds(client, seconds):
#just drives straight for given seconds
# input: async client, seconds
    global obstacle_detected
    global kidnapped
    await node.wait_for_variables()
    rt = RepeatedTimer(Ts, get_horizontal)
    # Set the motor speeds
    v = {
        "motor.left.target": [int(SPEED_LEFT)],
        "motor.right.target": [int(SPEED_RIGHT)],
    }
    await node.set_variables(v)
    start_time = time.monotonic()
    while time.monotonic() - start_time < seconds:
        await node.wait_for_variables({"prox.horizontal"})
        await node.wait_for_variables({"prox.ground.reflected"})
        
        #get_horizontal()
        await client.sleep(0.01)
        if obstacle_detected or kidnapped:
            print(f"kidnapped : ", kidnapped, f"obstacle: ", obstacle_detected)
            # Si un obstacle est détecté, arrêtez les moteurs
            v_stop = {
                "motor.left.target": [int(0)],
                "motor.right.target": [int(0)],
            }
            await node.set_variables(v_stop)
    
            #obstacle_detected = False  # Réinitialisez la variable d'obstacle détecté
            rt.stop()
            break  # Sortez de la boucle si un obstacle est détecté
        await asyncio.sleep(Ts)

    # Stop the motors
    v_stop = {
        "motor.left.target": [int(0)],
        "motor.right.target": [int(0)],
    }
    rt.stop()
    await node.set_variables(v_stop)
async def rotate_for_seconds(client, seconds, degrees):
##Rotates during a number of seconds
##input : async client, number of seconds, degrees (used to know if he turns clockwise or counter-clockwise
    global obstacle_detected, kidnapped
    await node.wait_for_variables()
    rt2 = RepeatedTimer(Ts, get_horizontal)
    if degrees >=0:
        v = {
            "motor.left.target": [int(-SPEED_LEFT)],
            "motor.right.target": [int(SPEED_RIGHT)],
        }
        await node.set_variables(v)
        start_time = time.monotonic()
        while time.monotonic() - start_time < seconds:
            await node.wait_for_variables({"prox.horizontal"})
            await node.wait_for_variables({"prox.ground.reflected"})
            
            await client.sleep(0.01)
            if obstacle_detected or kidnapped:
                # Si un obstacle est détecté, arrêtez les moteurs
                v_stop = {
                    "motor.left.target": [int(0)],
                    "motor.right.target": [int(0)],
                }
                await node.set_variables(v_stop)
                
                ##obstacle_detected = False  # Réinitialisez la variable d'obstacle détecté
                print(f"obstacle detected")
                break  # Sortez de la boucle si un obstacle est détecté
            await asyncio.sleep(Ts)
    
        # Stop the motors
        v_stop = {
            "motor.left.target": [int(0)],
            "motor.right.target": [int(0)],
        }
        rt2.stop()
        await node.set_variables(v_stop)
    else:
        v = {
            "motor.left.target": [int(SPEED_LEFT)],
            "motor.right.target": [int(-SPEED_RIGHT)],
        }
        await node.set_variables(v)
        start_time = time.monotonic()
        while time.monotonic() - start_time < seconds:
            await node.wait_for_variables({"prox.horizontal"})
            await node.wait_for_variables({"prox.ground.reflected"})
            
            await client.sleep(0.01)
            if obstacle_detected or kidnapped:
                print(f"obstacletranslation")
                # Si un obstacle est détecté, arrêtez les moteurs
                v_stop = {
                    "motor.left.target": [int(0)],
                    "motor.right.target": [int(0)],
                }
                await node.set_variables(v_stop)
                
                ##obstacle_detected = False  # Réinitialisez la variable d'obstacle détecté
                break  # Sortez de la boucle si un obstacle est détecté
            #await asyncio.sleep(Ts)
    
        # Stop the motors
        v_stop = {
            "motor.left.target": [int(0)],
            "motor.right.target": [int(0)],
        }
        rt2.stop()
        await node.set_variables(v_stop)

    
async def print_elapsed_time(seconds, interval=0.5):
##Testing function to know elapsed-time, no practical use apart from calibration, to delete later
    start_time = time.monotonic()
    while True:
        elapsed = time.monotonic() - start_time
        if elapsed > seconds:
            break
        print(f"Elapsed time: {elapsed:.4f} seconds")
        await asyncio.sleep(interval)
async def road_trip(nodes_to_visit, current_pos, initial_angle):
##Makes the robot visit nodes one after another
## input: nodes_to_visit:list of nodes containing [x,y] coordinates, 
##        current_pos : [x,y] coordinates of current position, 
##        initial_angle : current angle between thymio sagittal axis and x-axis
    global obstacle_detected, kidnapped
    previous_angle = 0;
    for i in range(len(nodes_to_visit)):
        if obstacle_detected or kidnapped: 
            break
        else: 
            if i == 0:
             dist, angle = compute_movement(current_pos, nodes_to_visit[i], initial_angle)
             previous_angle = angle + initial_angle
            else: 
             dist, angle = compute_movement(nodes_to_visit[i-1], nodes_to_visit[i], previous_angle)
             previous_angle = angle + previous_angle
            await go_to(dist, angle)
        
    
    
async def go_to(dist, angle):
## Makes the thymio rotate of given angle, then go straight for distance dist, only meant to be used through road_trip function
## Input: dist : distance to travel in mm
##        angle : angle to rotate before going forward

    secondstodrive = dist/(SPEED*thymio_speed_to_mms)
    if angle!= 0:
        disttorotate = abs(angle*rotation_factor)
        secondstorotate = disttorotate/(SPEED*thymio_speed_to_mms)
        rotate_task = asyncio.create_task(rotate_for_seconds(client, secondstorotate, np.sign(angle)))
        await rotate_task
    
    drive_task = asyncio.create_task(drive_for_seconds(client, secondstodrive))
    
    #print_task = asyncio.create_task(print_elapsed_time(secondstodrive))
    
    # Wait for both tasks to complete
    await drive_task
    
    #await print_task



### Example main program, all these lines are necessary

In [70]:
from tdmclient import ClientAsync, aw
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()
#Define the trip the robot has to make here : 
trip = ([0,100],[200,100], [200,300],[300,300], [300,0], [0,0])
trip2 = ([0,100], [0,200])
async def main():    
    global kidnapped
 # Start the driving task
    await road_trip(trip2, [0,0], -90)
    if kidnapped :
        v = {
            "leds.top":  [1, 1, 1]
        }
        
    else : v= {"leds.top":  [0, 0, 0]
        }

    aw(node.set_variables(v))
await main()
obstacle_detected = 0
kidnapped = 0
await node.unlock()

Distance: 100.0 degrees: 180.0
obstacle detected
kidnapped :  1 obstacle:  False


Quick explaination on how we computed the seconds required to rotate of x degrees


You can use the following formula to calculate the number of wheel rotations \(T\) required for one complete rotation:
$$T = \frac{\pi \times L}{\pi \times D} $$
We measured L = 9.5cm (distance between wheels) and D = 4.2 cm (wheel diameter), but this value is meant to be tuned
we want to know the number of degrees the wheels will turn for the vehicle to complete one full rotation, we can use the formula:
$$\text{Number of degrees} = T \times 360 $$

Substitute the value of \(T\) from the previous formula:
$$\text{Number of degrees} = \frac{L}{D} \times 360 = 814.286$$

Then from this we can compute the required distance to travel for a given wanted rotation: 
$$\frac{814.286}{95*\pi} = \frac{wanted rotation°}{distance} \implies distance = \frac{\text{wanted rotation°} \times 95 \pi}{814.286}$$
From which we compute the seconds required:
$$\text{Seconds of rotation} =  \frac{distance}{SPEED \times conversionfactor}$$
In practice, the values L and D have to be tuned.
